In [1]:
from fastai.vision.all import *
from fastai.learner import *
from fastai.data.all import *
from fastai.callback.tracker import SaveModelCallback
import pandas as pd
import matplotlib.pyplot as plt
from pathlib2 import Path
import numpy as np
import random
from torch.nn import MSELoss

In [2]:
%%time

df = pd.read_csv('data/examples_with_length.csv')
df.shape

CPU times: user 29.7 s, sys: 2.82 s, total: 32.5 s
Wall time: 32.5 s


(17937758, 11)

In [3]:
%%time
fn2features = pd.read_pickle('data/fn2feature.pkl')

CPU times: user 17.4 s, sys: 11.9 s, total: 29.3 s
Wall time: 1min 17s


In [4]:
df = df[((df.source_length < 70) & (df.target_length < 70) & (df.source_length > 25) & (df.target_length > 25))]
df.reset_index(drop=True, inplace=True)

In [5]:
df.target_length.max()

69

In [6]:
df.shape

(3336894, 11)

In [7]:
train_examples = df[df.set_name.isin(['train-clean-360', 'train-clean-100', 'dev-clean'])]
valid_examples = df[df.set_name == 'test-clean']

In [8]:
def prepare_features(row, col, pad_to=69, pad_left=False):
    ary = fn2features[row[col]]
    example = np.zeros((pad_to, 13))
    if pad_left:
        example[-ary.shape[0]:, :] = ary
    else: example[:ary.shape[0], :] = ary
    return example.astype(np.float32)

In [9]:
dataset_mean = -3
dataset_std = 12

def normalize_data(ary):
    return (ary - dataset_mean) / dataset_std

In [10]:
dss = Datasets(
    df,
    [lambda row: normalize_data(prepare_features(row, 'source_fn', pad_left=True)),
     lambda row: normalize_data(prepare_features(row, 'target_fn')),
     lambda row: normalize_data(prepare_features(row, 'target_fn'))],
    n_inp=2,
    splits = [train_examples.index, valid_examples.index]
)

In [11]:
BS = 2048
LR = 1e-3
NUM_WORKERS = 8

train_dl = DataLoader(dss.train, BS, NUM_WORKERS, shuffle=True)
valid_dl = DataLoader(dss.valid, BS, NUM_WORKERS)

dls = DataLoaders(train_dl, valid_dl)

In [12]:
# Got the following error while training:

# DataLoader worker (pid 2073) is killed by signal: Bus error. It is possible that dataloader's workers are out of shared memory. Please try to raise your shared memory limit.
# trying the solution I found here: https://github.com/pytorch/pytorch/issues/5040
# which is to execute
!sudo umount /dev/shm/ && sudo mount -t tmpfs -o rw,nosuid,nodev,noexec,relatime,size=50G shm /dev/shm

In [13]:
# bidirectional encoder, 1 layer, concatenate hidden state
class Model(Module):
    def __init__(self, hidden_size=25, num_layers_encoder=1):
        self.return_embeddings = False
        self.num_layers_encoder = num_layers_encoder
        self.hidden_size = hidden_size
        
        self.encoder= nn.LSTM(
            input_size=13,
            hidden_size=hidden_size,
            num_layers=self.num_layers_encoder,
            batch_first=True,
            dropout=0,
            bidirectional=True
        )
        self.decoder = nn.LSTM(
            input_size=2*hidden_size+13,
            hidden_size=2*hidden_size,
            num_layers=1,
            batch_first=True,
            dropout=0,
            bidirectional=False
        )
        self.lin = nn.Linear(2*hidden_size, 13)
            
    def forward(self, source_features, target_features):
        _, (embeddings, _) = self.encoder(source_features)
        
        embeddings = embeddings.view(self.num_layers_encoder, 2, source_features.shape[0], self.hidden_size)
        embeddings = torch.cat([embeddings[:, 0, :, :], embeddings[:, 1, :, :]], -1)
        if self.return_embeddings: return embeddings
        
        target_features = torch.cat((torch.zeros(target_features.shape[0], 1, 13).cuda(), target_features), 1)
        inputs = torch.cat(
            (
                target_features[:, :-1, :],
                embeddings.permute(1, 0, 2).repeat(1, target_features.shape[1]-1, 1)
            ), 2)
        x, _ = self.decoder(inputs, (embeddings, torch.zeros_like(embeddings)))
        return self.lin(x)

In [14]:
learn = Learner(dls.cuda(), Model().cuda(), loss_func=MSELoss(), lr=1e-3, opt_func=Adam)

In [ ]:
learn.fit(20, cbs=SaveModelCallback(fname='1e-3_Adam_tf', every_epoch=True), lr=1e-3)

epoch,train_loss,valid_loss,time
0,0.170540,0.171109,08:13
1,0.166538,0.167282,08:26
2,0.164858,0.165715,08:27
3,0.163919,0.164541,08:23
4,0.163171,0.163813,08:25
5,0.162741,0.163336,08:27
6,0.162202,0.162912,08:26


In [ ]:
learn.fit(20, cbs=SaveModelCallback(fname='1e-4_Adam_tf', every_epoch=True), lr=1e-4)

## Calculate embedding for each unique utterancein the dataset

In [ ]:
df_unique_utterances = df[df.set_name.isin(['train-clean-360', 'train-clean-100', 'dev-clean'])].drop_duplicates(['source_fn'])
df_unique_utterances.reset_index(drop=True, inplace=True)

In [ ]:
dss = Datasets(
    df_unique_utterances,
    [lambda row: normalize_data(prepare_features(row, 'source_fn', pad_left=True)),
     lambda row: normalize_data(prepare_features(row, 'target_fn')),
     lambda row: normalize_data(prepare_features(row, 'target_fn'))],
    n_inp=2,
)

In [ ]:
all_utterances_dl = DataLoader(dss, BS, NUM_WORKERS, shuffle=False)

In [ ]:
%%time

learn.model.return_embeddings = True
learn.model.train(False)

all_embeddings = []

with torch.no_grad():    
    for batch in all_dl:
        embeddings = learn.model(batch[0].cuda())
        all_embeddings.append(embeddings.detach().cpu().squeeze(0))

In [ ]:
all_embeddings = torch.cat(all_embeddings)

In [ ]:
all_embeddings.shape

In [ ]:
learn.model.return_embeddings = False

with torch.no_grad():
    for i, batch in enumerate(all_utterances_dl):
        outputs = learn.model(batch[0].cuda(), batch[1].cuda())
        break

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(outputs[31].cpu().numpy().T[:, :20])
axs[1].imshow(batch[1][31].cpu().numpy().T[:, :20])

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(outputs[0].cpu().numpy().T[:, :20])
axs[1].imshow(batch[1][0].cpu().numpy().T[:, :20])

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(outputs[30].cpu().numpy().T[:, :20])
axs[1].imshow(batch[1][30].cpu().numpy().T[:, :20])

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(outputs[15].cpu().numpy().T[:, :20])
axs[1].imshow(batch[1][15].cpu().numpy().T[:, :20])

In [ ]:
# %%time

# word2row_idxs_unique_utterances = defaultdict(empty_list)

# for idx, row in df_unique_utterances.iterrows():
#     word2row_idxs_unique_utterances[row.source_word].append(idx)
    
# pd.to_pickle(word2row_idxs_unique_utterances, 'word2row_idxs_unique_utterances_max_length.pkl')

In [ ]:
word2row_idxs_unique_utterances = pd.read_pickle('word2row_idxs_unique_utterances_max_length.pkl')

In [ ]:
word2embedding = {}

for k, v in word2row_idxs_unique_utterances.items():
    word2embedding[k] = all_embeddings[np.array(v)].mean(0)

In [ ]:
word2embedding_without_nans= {}
nans_encountered = 0
for k, v in word2embedding.items():
    if k in vocab and k == k and (not np.isnan(v.numpy()).any()):
        word2embedding_without_nans[k] = v.numpy()
    else: nans_encountered += 1

print(f'Encountered rows with nan values: {nans_encountered}')

In [ ]:
from utils import Embeddings

In [ ]:
e = Embeddings(
    np.array(list(word2embedding_without_nans.values())),
    [w.lower() for w in list(word2embedding_without_nans.keys())]
)

In [ ]:
for w in ['fast', 'lost', 'small', 'true', 'crazy', 'slow']:
    print(f'{w}: {e.nn_words_to(e[w])}')

## Evaluating embeddings using [word-embeddings-benchmarks](https://github.com/kudkudak/word-embeddings-benchmarks)

In [ ]:
import logging
from six import iteritems
from web.datasets.similarity import fetch_MEN, fetch_WS353, fetch_SimLex999
from web.embeddings import fetch_GloVe
from web.evaluate import evaluate_similarity
from web.embedding import Embedding, Vocabulary
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
tasks = {
    "MEN": fetch_MEN(),
    "WS353": fetch_WS353(),
    "SIMLEX999": fetch_SimLex999()
}

In [ ]:
our_embeddings = Embedding(
    Vocabulary([w.lower() for w in list(word2embedding_without_nans.keys())]),
    np.array(list(word2embedding_without_nans.values()))
)

speech2vec = KeyedVectors.load_word2vec_format('../speech2vec-pretrained-vectors/speech2vec/50.vec', binary=False) 
speech2vec_embeddings = Embedding(Vocabulary(list(speech2vec.vocab.keys())), speech2vec.vectors)

In [ ]:
for name, data in iteritems(tasks):
    print("Spearman correlation of scores on {} {}".format(name, evaluate_similarity(our_embeddings, data.X, data.y)))

In [ ]:
for name, data in iteritems(tasks):
    print("Spearman correlation of scores on {} {}".format(name, evaluate_similarity(speech2vec_embeddings, data.X, data.y)))